In [115]:
import os
import findspark

In [116]:
findspark.init()

In [117]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, explode, regexp_replace, split, col, size, array_contains, isnan, when, count, array, reverse, udf, unix_timestamp, from_unixtime, date_format, format_number, length, first
from pyspark.sql.types import ArrayType, StringType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, skew
import math
import folium
# from geopy.geocoders import Nominatim
from itertools import chain

In [118]:
spark_url = 'local'
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Project')\
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")\
        .getOrCreate()
spark.conf.set("spark.sql.csv.parser.multiLine", "true")

In [119]:
# path = '/content/drive/MyDrive/traffy/bangkok_traffy.csv'
path = 'bangkok_traffy.csv'
df = spark.read.option("multiLine", "true").csv(path, header=True, inferSchema=True)
df.show(5)

+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|               type|        organization|             comment|               photo|         photo_after|            coords|             address|subdistrict|district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|                 {}|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13.79386|1867 จรัญสนิทวงศ์...|    บางพลัด| บางพล

In [120]:
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|ticket_id|type|organization|comment|photo|photo_after|coords|address|subdistrict|district|province|timestamp|state|  star|count_reopen|last_activity|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|     2413|1550|        2640|   3911| 2089|      85624|  2019|   4433|       2089|    2092|    2393|     2233| 2026|164087|      120042|         2558|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+



In [121]:
# Filter the DataFrame to include only rows with null values in the "column_name" column
def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row
    
ticket_id_null_df = df.filter(df.ticket_id.isNull())
coords_null_df = df.filter(df.coords.isNull())
address_null_df = df.filter(df.address.isNull())
print(check_first_null(ticket_id_null_df), "\n")
print(check_first_null(coords_null_df), "\n")
print(check_first_null(address_null_df), "\n")

Row(ticket_id=None, type='{ถนน}', organization='สำนักงาน ป.ป.ท.,เขตจอมทอง,ฝ่ายเทศกิจ เขตจอมทอง,ผอ.เขตจอมทอง (นายณัฐพงษ์),กลุ่มกรุงธนเหนือ (รองปลัดฯ เฉลิมพล)', comment=None, photo='https://storage.googleapis.com/traffy_public_bucket/TeamChadChart/corruption_photo2.png', photo_after='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-06/e9596093de70ae8abacd6574f26a2d0f4466fe9f.jpg', coords='100.45568,13.69103', address=None, subdistrict='บางขุนเทียน', district='จอมทอง', province='กรุงเทพมหานคร', timestamp='2022-06-09 23:34:34.98044+00', state='เสร็จสิ้น', star='5', count_reopen=None, last_activity='2022-06-10 11:02:34.607728+00') 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', organization=None, comment='"เคยดีใจมีสายสีน้ำเงินสถานี""แยกไฟฉาย""', photo=None, photo_after=None, coords=None, address=None, subdistrict=None, district=None, province=None, timestamp=None, state=None, star=None, count_reopen=None, last_activity=None) 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', 

From above cells give us the first row with null value from each column selected (ticket_id, coords, address) to tell some relationship of those null value.
1. The ticket_id is null when the state='เสร็จสิ้น', it's mean we can drop this column significantly.
2. The address has null value 2 times more than coords. In the first null row we can see both of them are null. So it might tell that if no coords, no address too and not vice versa. We'll check in next step. 

In [122]:
sub = ['coords', 'address']
df = df.dropna(how='all', subset=sub)
sub_null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sub])
sub_null_counts.show()

+------+-------+
|coords|address|
+------+-------+
|     1|   2415|
+------+-------+



As we can see, the null coords most dissapear. So, we don't need to map any address to coords and we can also drop all remained null column.

In [123]:
df = df.dropna(how='all', subset=['coords'])
df = df.dropna(how='all', subset=['address'])

From this step, we'll use df_use to be a data for ML

In [124]:
# change state to boolean 
# df_use = df.withColumn('state', when(df.state == 'เสร็จสิ้น', 1).otherwise(0))
df_use = df.filter(df.state == 'เสร็จสิ้น')

# change type to list
df_use = df_use.withColumn("type", split(regexp_replace("type", "[{}]", ""), ","))
df_use = df_use.dropna(how='all', subset=['type'])

# change coords to pair and swap them into format [latitude, longtitude]
flatten = udf(lambda x: list(chain.from_iterable(x)), ArrayType(StringType()))
df_use = df_use.withColumn('coords', array(reverse(split(df.coords, ','))))
df_use = df_use.withColumn('coords', flatten('coords'))

# change timestamp and last_activity
df_use = df_use.withColumn('timestamp', from_unixtime(unix_timestamp(col('timestamp'), 'yyyy-MM-dd HH:mm')))
df_use = df_use.withColumn('last_activity', from_unixtime(unix_timestamp(col('last_activity'), 'yyyy-MM-dd HH:mm')))
df_use = df_use.withColumn('time_diff', (unix_timestamp(col('last_activity')) - unix_timestamp(col('timestamp'))) / 86400)
df_use = df_use.withColumn('time_diff', format_number(col('time_diff'), 0))
df_use.show(5)

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+-------------------+---------+----+------------+-------------------+---------+
|  ticket_id|                type|        organization|             comment|               photo|         photo_after|              coords|             address|subdistrict|district|     province|          timestamp|    state|star|count_reopen|      last_activity|time_diff|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+-------------------+---------+----+------------+-------------------+---------+
|2021-FYJTFP|         [ความสะอาด]|          เขตบางซื่อ|             ขยะเยอะ|https://storage.g...|                null|[13.81865, 100.53...|12/14 ถนน กรุงเทพ...|       null|    nu

In [125]:
df_use.groupBy(col('star')).count().show()

+----+-----+
|star|count|
+----+-----+
|   3| 9949|
|null|97721|
|   5|45519|
|   1|13017|
|   4|20144|
|   2| 4456|
+----+-----+



In [126]:
def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row
    
star_null_df = df_use.filter(df_use.star.isNull())
print(check_first_null(star_null_df), "\n")

Row(ticket_id='2021-FYJTFP', type=['ความสะอาด'], organization='เขตบางซื่อ', comment='ขยะเยอะ', photo='https://storage.googleapis.com/traffy_public_bucket/attachment/2021-09/3063e748259afbb7171467e19b92e9cc1f1a5826.jpg', photo_after=None, coords=['13.81865', '100.53084'], address='12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย', subdistrict=None, district=None, province='กรุงเทพมหานคร', timestamp='2021-09-03 12:51:00', state='เสร็จสิ้น', star=None, count_reopen=None, last_activity='2022-06-04 15:34:00', time_diff='274') 



In [127]:
first_element = udf(lambda x: x[0], StringType())
# df_exploded = df_use.withColumn('type', first_element(df_use['type']))
df_exploded = df_use.withColumn('type', explode(df_use['type']))
df_exploded = df_exploded.filter("type != ''")

df_exploded.show()

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+-------------------+---------+----+------------+-------------------+---------+
|  ticket_id|       type|        organization|             comment|               photo|         photo_after|              coords|             address|subdistrict|district|     province|          timestamp|    state|star|count_reopen|      last_activity|time_diff|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+-------------------+---------+----+------------+-------------------+---------+
|2021-FYJTFP|  ความสะอาด|          เขตบางซื่อ|             ขยะเยอะ|https://storage.g...|                null|[13.81865, 100.53...|12/14 ถนน กรุงเทพ...|       null|    null|กรุงเทพมหานคร|2021-09-03 12:51:00

In [128]:
df_t = df_exploded.withColumn("latitude", col("coords")[0]) \
       .withColumn("longitude", col("coords")[1]) \
       .drop("coords")
df_t = df_t.withColumn("latitude", col("latitude").cast('double'))
df_t = df_t.withColumn("longitude", col("longitude").cast('double'))

In [129]:
null_latitude = df_t.filter(df_t.latitude.isNull())
null_longitude = df_t.filter(df_t.longitude.isNull())
print(check_first_null(null_latitude), "\n")
print(check_first_null(null_longitude), "\n")

No null values in DataFrame[ticket_id: string, type: string, organization: string, comment: string, photo: string, photo_after: string, address: string, subdistrict: string, district: string, province: string, timestamp: string, state: string, star: string, count_reopen: string, last_activity: string, time_diff: string, latitude: double, longitude: double]. 

No null values in DataFrame[ticket_id: string, type: string, organization: string, comment: string, photo: string, photo_after: string, address: string, subdistrict: string, district: string, province: string, timestamp: string, state: string, star: string, count_reopen: string, last_activity: string, time_diff: string, latitude: double, longitude: double]. 



In [130]:
drop_list = ['type', 'latitude', 'longitude', 'subdistrict', 'district', 'timestamp', 'last_activity', 'organization', 'star', 'time_diff']
for e in drop_list:
    df_t = df_t.dropna(how='all', subset=[e])

## Classification

In [131]:
env = os.getenv('HADOOP_HOME')
print(env)

C:\hadoop-3.3.5


In [132]:
def train_random_forest(df, label_col, numeric_col,  quantitative_col):
    stages = []

    # For label
    label_indexer = StringIndexer(inputCol=label_col, outputCol='label')
    indexer_model = label_indexer.fit(df)
    df = indexer_model.transform(df)

    # For features
    for column_name in quantitative_col:
        stringIndexer = StringIndexer(inputCol=column_name, outputCol=column_name+ "_index")
        stages.append(stringIndexer)
        oneHotEncoder = OneHotEncoder(inputCol=column_name+ "_index", outputCol=column_name + "_onehot")
        stages.append(oneHotEncoder)

    quantitative_cols = [s + "_onehot" for s in quantitative_col]
    assembler = VectorAssembler(inputCols=numeric_col + quantitative_cols, outputCol='features')
    stages.append(assembler)

    pipeline = Pipeline(stages=stages)
    preprocessed_data = pipeline.fit(df).transform(df)

    # Split data into training and testing sets
    # train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3], seed=123)

    split_fraction = {'1': 0.7, '2': 0.7, '3': 0.7, '4': 0.7, '5': 0.7}
    train_data = preprocessed_data.sampleBy(label_col, fractions=split_fraction, seed=123)
    # train_data.groupBy(col(label_col)).count().show()
    test_data = preprocessed_data.subtract(train_data)
    # test_data.groupBy(col(label_col)).count().show()

    # Train Decision Tree classifier
    dt = RandomForestClassifier(labelCol='label', featuresCol='features', maxDepth=6)
    dt_model = dt.fit(train_data)
    
    # Make predictions on test set
    predictions = dt_model.transform(test_data)
    label_reverse = IndexToString(inputCol="prediction", outputCol="predicted_result", labels=indexer_model.labels)
    predictions_res = label_reverse.transform(predictions)

    # Evaluate performance
    evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
    accuracy = evaluator.evaluate(predictions)
    print('Accuracy:', accuracy)

    # experiment_name = "traffy_predict_star"
    # mlflow.set_experiment(experiment_name)

    # with mlflow.start_run():
    #     # log param
    #     mlflow.log_param('maxDepth', 6)
    #     mlflow.log_param('seed', 123)

    #     # log metric
    #     mlflow.log_metric('accuracy', accuracy)

    #     # save model
    #     mlflow.spark.save_model(dt_model, 'random_forest_model')
    #     mlflow.end_run()

    return predictions_res, dt_model

In [145]:
numeric_col = []
quantitative_col = ['type', 'star', 'time_diff', 'subdistrict', 'district']
df_select = df_t.select(numeric_col + quantitative_col)
try:
    predictions_res, training_model = train_random_forest(df_select, 'star', numeric_col,  quantitative_col)
except Exception as e:
    print(e)

Accuracy: 0.9105613131252621


In [146]:
predictions_res.groupBy(col('predicted_result')).count().show()

+----------------+-----+
|predicted_result|count|
+----------------+-----+
|               3| 2112|
|               5| 7947|
|               1| 2878|
|               4| 3756|
+----------------+-----+



## Regression

In [135]:
def train_linear_regression(df, label_col, numeric_col,  quantitative_col):
    stages = []
    df_select = df.select(numeric_col + quantitative_col)

    # For label
    label_indexer = StringIndexer(inputCol=label_col, outputCol='label')
    indexer_model = label_indexer.fit(df_select)
    df_select = indexer_model.transform(df_select)

    # For features
    for column_name in quantitative_col:
        stringIndexer = StringIndexer(inputCol=column_name, outputCol=column_name+ "_index")
        stages.append(stringIndexer)
        oneHotEncoder = OneHotEncoder(inputCol=column_name+ "_index", outputCol=column_name + "_onehot")
        stages.append(oneHotEncoder)

    quantitative_cols = [s + "_onehot" for s in quantitative_col]
    assembler = VectorAssembler(inputCols=numeric_col + quantitative_cols, outputCol='features')
    stages.append(assembler)

    pipeline = Pipeline(stages=stages)
    preprocessed_data = pipeline.fit(df_select).transform(df_select)

    # Split data into training and testing sets
    train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3], seed=123)

    # Train Decision Tree classifier
    poisson = GeneralizedLinearRegression(family='poisson', link='log', maxIter=10, regParam=0.3)
    dt_model = poisson.fit(train_data, label_col)
    predictions = dt_model.transform(test_data)

    # Make predictions on test set
    label_reverse = IndexToString(inputCol="prediction", outputCol="predicted_result", labels=indexer_model.labels)
    predictions_res = label_reverse.transform(predictions)

    # Evaluate the performance of the model using mean squared error
    from pyspark.ml.evaluation import RegressionEvaluator
    evaluator = RegressionEvaluator(predictionCol='prediction', labelCol=label_col, metricName='mse')
    mse = evaluator.evaluate(predictions)
    print('Mean Squared Error:', mse)

    return predictions_res

In [159]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/', methods=['GET'])
def hello():
    return "Hello Guys"

@app.route('/predict', methods=['POST'])
def predict_star():
    data = request.json  
    df = pd.DataFrame(data, index=[0])
    spark_pred = SparkSession.builder.getOrCreate()
    spark_df = spark_pred.createDataFrame(df)

    label_indexer = StringIndexer(inputCol='star', outputCol='label')
    indexer_model = label_indexer.fit(df_select)
    indexer_model_trans = indexer_model.transform(spark_df)

    indexer = StringIndexer(inputCols=['type', 'time_diff', 'subdistrict', 'district'], outputCols=['type_indexed', 'time_diff_indexed', 'subdistrict_indexed', 'district_indexed'])
    indexed_df = indexer.fit(df_select).transform(indexer_model_trans)
    encoder = OneHotEncoder(inputCols=['type_indexed', 'time_diff_indexed', 'subdistrict_indexed', 'district_indexed'], outputCols=['type_encoded', 'time_diff_encoded', 'subdistrict_encoded', 'district_encoded'])
    encoded_df = encoder.fit(indexed_df).transform(indexed_df)
    assembler = VectorAssembler(inputCols=['type_encoded', 'time_diff_encoded', 'subdistrict_encoded', 'district_encoded'], outputCol='features')
    final_df = assembler.transform(encoded_df)
    
    predictions = training_model.transform(final_df)
    predict_label_reverse = IndexToString(inputCol="prediction", outputCol="predicted_result", labels=indexer_model.labels)
    predictions_res = predict_label_reverse.transform(predictions)

    predicted_star = predictions_res.select('predicted_result').first()[0]
    response = {'predicted_star': predicted_star}

    return jsonify(response)

if __name__ == '__main__':
    app.run()

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [16/May/2023 18:41:25] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2023 18:41:25] "POST /predict HTTP/1.1" 200 -


In [160]:
spark.stop()